In [1]:
import requests
import pandas as pd 
from tqdm.notebook import tqdm

In [2]:
api_key = None

In [3]:
markets = [
    'de_DE',
    'en_US',
]

headers = {
    'accept': 'application/json',
    'x-api-key': api_key,
}

In [4]:
data = {
    'market_id': [],
    'model_id': [],
    'type_class': [],
    'name': [],
    'short_name': [],
    'brand': [],
    'baumuster': [],
    'class': [],
    'body': [],
    'model_year': [],
    'product_group': [],
    'all_terrain': [],
    'currency': [],
    'net_price': [],
    'price': [],
    'steering_position': [],
    'conf_url': [],
}

save_all_data = {}

for market_id in markets:
    url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/bodies'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        bodies = response.json()
    else:
        raise Exception(f'Body request {url} failed')
    
    save_all_data[market_id] = {'bodies': bodies}
    bodies = [body['bodyId'] for body in bodies]

    for body_id in tqdm(bodies):
        url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models?bodyId={body_id}'

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            models = response.json()
        else:
            raise Exception(f'Models request {url} failed')

        for model in models:
            data['market_id'].append(market_id)
            data['model_id'].append(model['modelId'])
            data['type_class'].append(model['typeClass'])
            data['name'].append(model['name'])
            data['short_name'].append(model['shortName'])
            data['brand'].append(model['brand']['name'])
            data['baumuster'].append(model['baumuster'])
            data['class'].append(model['vehicleClass']['className'])
            data['body'].append(model['vehicleBody']['bodyName'])
            data['model_year'].append(model['modelYear'])
            data['product_group'].append(model['productGroup'])
            data['all_terrain'].append(model['allTerrain'])
            data['currency'].append(model['priceInformation']['currency'])
            data['net_price'].append(model['priceInformation']['netPrice'])
            data['price'].append(model['priceInformation']['price'])
            data['steering_position'].append(model['steeringPosition'])
            data['conf_url'].append(model['_links']['configurations'])

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [5]:
df = pd.DataFrame(data)

In [6]:
df_ev = pd.concat([
    df.query('name.str.contains("EQ")'),
    df.query('name.str.contains(" E ")'),
    df.query('name.str.contains("HYBRID")'),
]).drop_duplicates()


In [7]:
df_ev.to_csv('mercedes_ev.csv', index=False)

In [8]:
configurations = []
for _, market_id, model_id, url in tqdm(df_ev[['market_id', 'model_id', 'conf_url']].itertuples()):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        confs = response.json()
    else:
        print("Request failed with status code:", response.status_code)
        confs = {'configurationId': None}

    configurations.append(confs['configurationId'])

0it [00:00, ?it/s]

Request failed with status code: 500
Request failed with status code: 500
Request failed with status code: 500
Request failed with status code: 500


In [9]:
df_ev['conf_id'] = configurations

In [22]:
api_key = 'd26895a7-ecc3-44e9-ad26-400087ce44ea'

In [23]:
img_ext_urls = []
img_int_urls = []
for (_, market_id, model_id, conf_id) in tqdm(df_ev[['market_id', 'model_id', 'conf_id']].itertuples()):
    url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models/{model_id}/configurations/{conf_id}/images/vehicle?perspectives=EXT020%2CINT1&roofOpen=false&night=false&background=true'
    response = requests.get(url, headers={'x-api-key': api_key})
    if response.status_code == 200:
        imgs = response.json()
    else:
        print("Request failed with status code:", response.status_code)
        imgs = {'vehicle': {'EXT020': {'url': None}, 'INT1': {'url': None}}}
    
    img_ext_urls.append(imgs['vehicle']['EXT020']['url'])
    img_int_urls.append(imgs['vehicle']['INT1']['url'])

0it [00:00, ?it/s]

Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
Request failed with status code: 400
R

In [24]:
df_ev['img_ext_url'] = img_ext_urls
df_ev['img_int_url'] = img_int_urls

In [25]:
df_ev['img_ext_url'].isna().sum()

93

In [27]:
(~df_ev['img_int_url'].isna()).sum()

15

In [33]:
from PIL import Image
from io import BytesIO

img_ext_path = []
img_int_path = []

img_num = 0

for _, url_ext, url_int in tqdm(df_ev[['img_ext_url', 'img_int_url']].itertuples()):
    img_num += 1

    if url_ext is not None:
        response = requests.get(url_ext)
        image_content = response.content

        # Create an Image object from the image content
        image = Image.open(BytesIO(image_content))
        path = f'imgs/{img_num}_ext.jpg'
        image.save(path)
        img_ext_path.append(path)
    else:
        img_ext_path.append(None)


    if url_int is not None:
        response = requests.get(url_int)
        image_content = response.content

        # Create an Image object from the image content
        image = Image.open(BytesIO(image_content))
        path = f'imgs/{img_num}_int.jpg'
        image.save(path)
        img_int_path.append(path)
    else:
        img_int_path.append(None)

df_ev['img_ext_path'] = img_ext_path
df_ev['img_int_path'] = img_int_path

0it [00:00, ?it/s]

In [34]:
df_ev.to_csv('mercedes_ev.csv', index=False)

In [36]:
df_ev.columns

Index(['market_id', 'model_id', 'type_class', 'name', 'short_name', 'brand',
       'baumuster', 'class', 'body', 'model_year', 'product_group',
       'all_terrain', 'currency', 'net_price', 'price', 'steering_position',
       'conf_url', 'conf_id', 'img_ext_url', 'img_int_url', 'img_ext_path',
       'img_int_path'],
      dtype='object')

In [38]:
df_ev[['market_id', 'model_id', 'name', 'short_name', 'brand', 'class', 'body', 'all_terrain', 'currency', 'net_price', 'price', 'img_ext_path', 'img_int_path']].to_csv('mercedes_ev_llm.csv', index=None)

# Experiments

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/vehicle-models/markets/de_DE/baumuster/2437011?modelYear=805'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    details = response.json()
else:
    print("Request failed with status code:", response.status_code)


In [ ]:
api_key

In [ ]:
language = 'en'

url = f'https://api.mercedes-benz.com/configurator/v2/markets?language={language}'

headers = {
    'accept': 'application/json',
    'x-api-key': api_key,
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    markets = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
market_id = markets[0]['marketId']

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/bodies'

response = requests.get(url, headers=headers)
if response.status_code == 200:
    bodies = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
body_id = bodies[2]['bodyId']

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models?bodyId={body_id}'

response = requests.get(url, headers=headers)
if response.status_code == 200:
    models = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
model_id = models[0]['modelId']

In [ ]:
url = models[0]['_links']['configurations']

response = requests.get(url, headers=headers)
if response.status_code == 200:
    configurations = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
url = configurations['_links']['selectables']
response = requests.get(url, headers=headers)
if response.status_code == 200:
    selectables = response.json()
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
conf_id = configurations['configurationId']

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/markets/{market_id}/models/{model_id}/configurations/{conf_id}/images/vehicle?perspectives=EXT020%2CINT1&roofOpen=false&night=false&background=true'

response = requests.get(url, headers={'x-api-key': api_key})
if response.status_code == 200:
    imgs = response.json()
else:
    print("Request failed with status code:", response.status_code)


In [ ]:
imgs['vehicle']['EXT020']['url']

In [ ]:
vehicle_id = configurations['vehicleId']

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/stocksearch/vehicles/{vehicle_id}'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    stocks = response.json()
else:
    print("Request failed with status code:", response.status_code)


In [ ]:
api_key

In [ ]:
!curl -X GET "$url" -H "accept: application/json" -H "x-api-key:$api_key"

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/vehicle-models/markets/{market_id}'

response = requests.get(url, headers={'x-api-key': api_key})
if response.status_code == 200:
    detailed = response.json()
else:
    print("Request failed with status code:", response.status_code)


In [ ]:
models[0]

In [ ]:
baumuster = models[0]['baumuster']
model_year = models[0]['modelYear']

In [ ]:
url = f'https://api.mercedes-benz.com/configurator/v2/vehicle-models/markets/{market_id}/baumuster/{baumuster}?modelYear={model_year}'
response = requests.get(url, headers=headers)
if response.status_code == 200:
    detailed = response.json()
else:
    print("Request failed with status code:", response.status_code)
